# Metody Kryptografii w analizie danych

# Kryptosystemy częściowo homomorficzne

## RSA

Mamy klucz publiczny $(n,e)$ i klucz prywatny $(n,d)$. Szyfrujemy tym samym kluczem publicznym dwie wiadomości $m_1$ i $m_2$. Otrzymane szyfrogramy są postaci:
$$c_1=m_1^e\mod n\qquad c_2=m_2^e\mod n.$$

Jeżeli teraz spróbujemy zdeszyfrować kluczem prywatnym iloczyn otrzymanych szyfrogramów, to otrzymamy
$$(c_1c_2)^d\mod n=c_1^dc_2^d\mod n=(c_1^d\mod n)(c_2^d\mod n)=m_1m_2$$

Otrzymaliśmy zatem, że
$$D\big(E(m_1)E(m_2)\big)=m_1m_2$$
czyli **RSA jest częściowo homomorficzny ze względu na mnożenie**.

## Ćwiczenie 1.

Korzystając z implementacji RSA z ostatnich zajęć sprawdź czy są limity dla liczby homomorficznych operacji mnożenia (tzn. czy od jakiejś liczby operacji na szyfrogramach zaczynamy otrzymywać błędne deszyfrowanie).

In [11]:
from Crypto.Util import number
import random
from math import gcd

MIN_DIST = 2 ** 64
MIN_E = 2 ** 16


def gen_rsa_keys(n_length=2048):
    min_dist = 2 ** int(0.1 * n_length) + 1
    min_e = 2 ** int(0.05 * n_length) + 1
    p = number.getPrime(n_length)
    q = number.getPrime(n_length)
    while abs(p - q) < min_dist:
        q = number.getPrime(n_length)
    n = p * q
    euler_n = (p - 1) * (q - 1)
    e = random.randrange(min_e, euler_n)
    while gcd(e, euler_n) != 1:
        e = random.randrange(2, euler_n)
    d = pow(e, -1, euler_n)
    return (n, e), (n, d)


def rsa_encrypt(message, public_key):
    (n, e) = public_key
    return pow(message, e, n)


def rsa_decrypt(cipher, private_key):
    (n, d) = private_key
    return pow(cipher, d, n)

In [3]:
pub_key, priv_key = gen_rsa_keys(1024)

In [4]:
message1 = 2137
message2 = 420
ciphertext1 = rsa_encrypt(message1, pub_key)
ciphertext2 = rsa_encrypt(message2, pub_key)
ciphertext_product = ciphertext1 * ciphertext2
homomorphic_product = rsa_decrypt(ciphertext_product, priv_key)

print(f"Regular product: {message1 * message2}")
print(f"Homomorphic product: {homomorphic_product}")

Regular product: 897540
Homomorphic product: 897540


In [9]:
message = 10
regular_product = message
ciphertext = rsa_encrypt(message, pub_key)
ciphertext_product = ciphertext
for i in range(1000000):
    regular_product *= message
    ciphertext_product *= ciphertext
    homomorphic_product = rsa_decrypt(ciphertext_product, priv_key)
    if regular_product != homomorphic_product:
        print(f"Product difference at iteration {i}:")
        print(f"Regular product: {regular_product}")
        print(f"Homomorphic product: {homomorphic_product}")
        break

Product difference at iteration 615:
Regular product: 100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Homomorphic product: 820426917990511330162176793847189816548486988998299072788844590593426906518536353840972272392172682761234978182215322659373658026049800087227482040723763653393614940070892387186026200878392163582805615143256561476683041115939537029755601091411465145590729317913289762825517517897736225452924902126835050049

## Kryptosystem Pailliera

### Generowanie kluczy
- wybieramy losowo liczby pierwsze $p$, $q$ o tej samej długości w zapisie dziesiętnym i obliczamy $n=pq$, $g=n+1$, $\lambda=\phi(n)=(p-1)(q-1)$ oraz  $\mu =\phi (n)^{-1}{\mod {n}}$.
- kluczem publicznym jest $(n,g)$ a kluczem prywatnym $(\lambda,\mu)$


### Szyfrowanie
Szyfrujemy liczbę $0\leq m<n$. Wybieramy losowe $r<n$ względnie pierwsze z $n$ i obliczamy szyfrogram $$c=g^{m}\cdot r^{n}{\mod {n}}^{2}.$$


### Deszyfrowanie
Obliczamy $$m=\mu L(c^{\lambda }{\bmod {n}}^{2}){\bmod {n}},$$ gdzie  $L(x)$ to największa liczba naturalna $\nu$ taka, że $x-1\geq \nu n$.

Pomimo czynnika losowego przy szyfrowaniu, kryptosystem Pailliera ma własność $$D\big(E(m_{1},r_{1})\cdot E(m_{2},r_{2})\big)=m_{1}+m_{2}$$
a zatem jest **częściowo homomorficzny ze względu na dodawanie**.

## Ćwiczenie 2.

Zaimplementuj kryptosystem Pailliera. Sprawdź, czy są limity homomorficznych operacji dodawania.

In [24]:
def gen_paillier_keys(n_length=2048):
    min_dist = 2 ** int(0.1 * n_length) + 1
    p = number.getPrime(n_length)
    q = number.getPrime(n_length)
    while abs(p - q) < min_dist:
        q = number.getPrime(n_length)
    n = p * q
    g = n + 1
    lambda_ = (p - 1) * (q - 1)
    mi = pow(lambda_, -1, n)
    return (n, g), (lambda_, mi)


def paillier_encrypt(message, public_key):
    (n, g) = public_key
    r = n
    while gcd(n, r) != 1:
        r = random.randint(2, n)
    return (pow(g, message, n ** 2) * pow(r, n, n ** 2)) % (n ** 2)
    # return ((g ** message) * (r ** n)) % (n ** 2)


def paillier_decrypt(ciphertext, public_key, private_key):
    (lambda_, mi) = private_key
    (n, g) = public_key
    x = pow(ciphertext, lambda_, n ** 2)
    lx = (x - 1) // n 
    return (mi * lx) % n

In [26]:
p_pub_key, p_priv_key = gen_paillier_keys(10)

message = 2137
ciphertext = paillier_encrypt(message, p_pub_key)
decrypted = paillier_decrypt(ciphertext, p_pub_key, p_priv_key)

decrypted

2137

In [29]:
message1 = 2137
message2 = 420
ciphertext1 = paillier_encrypt(message1, p_pub_key)
ciphertext2 = paillier_encrypt(message2, p_pub_key)
ciphertext_product = ciphertext1 * ciphertext2
homomorphic_sum = paillier_decrypt(ciphertext_product, p_pub_key, p_priv_key)

print(f"Regular sum: {message1 + message2}")
print(f"Homomorphic sum: {homomorphic_product}")


Regular sum: 2557
Homomorphic sum: 2557


Kryptosystem nazywamy *w pewnym sensie homomorficznym*, jeżeli operacjami na samych szyfrogramach $E(m_1),...,E(m_k)$ jesteśmy w stanie obliczyć pewne określone kombinacje dodawania i mnożenia oryginalnych wiadomości $m_1,...,m_k$.

## Ćwiczenie 3.

Wykorzystując RSA i Pailliera zaimplementuj kryptosystem, który będzie w stanie obliczyć $(m_1m_2+m_3)\cdot m_4$. Wszystkie działania muszą się odbywać na danych zaszyfrowanych (nie możemy ujawnić chmurze żadnej wiadomości $m_1$, $m_2$, $m_3$, $m_4$). Czy jesteśmy w stanie w ten sposób skonstruować kryptosystem w pewnym sensie homomorficzny?

In [33]:
m1 = 3
m2 = 6
m3 = 11
m4 = 8

p_pub_key, p_priv_key = gen_paillier_keys(10)
r_pub_key, r_priv_key = gen_rsa_keys(10)

m1_rsa = rsa_encrypt(m1, r_pub_key)
m2_rsa = rsa_encrypt(m2, r_pub_key)
# client -> server (m1_rsa, m2_rsa)
m1_times_m2_rsa = m1_rsa * m2_rsa
# server -> client m1_times_m2_rsa
m1_times_m2 = rsa_decrypt(m1_times_m2_rsa, r_priv_key)
m1_times_m2_paillier = paillier_encrypt(m1_times_m2, p_pub_key)
m3_paillier = paillier_encrypt(m3, p_pub_key)
# client -> server (m1_times_m2_paillier, m3_paillier)
m1_times_m2_plus_m3_paillier = m1_times_m2_paillier * m3_paillier
# server -> client m1_times_m2_plus_m3_paillier
m1_times_m2_plus_m3 = paillier_decrypt(m1_times_m2_plus_m3_paillier, p_pub_key, p_priv_key)
m1_times_m2_plus_m3_rsa = rsa_encrypt(m1_times_m2_plus_m3, r_pub_key)
m4_rsa = rsa_encrypt(m4, r_pub_key)
# client -> server (m1_times_m2_plus_m3_rsa, m4_rsa)
m1_times_m2_plus_m3_times_m4_rsa = m1_times_m2_plus_m3_rsa * m4_rsa
# server -> client m1_times_m2_plus_m3_times_m4_rsa
m1_times_m2_plus_m3_times_m4 = rsa_decrypt(m1_times_m2_plus_m3_times_m4_rsa, r_priv_key)

print(f"Regular result: {(m1 * m2 + m3) * m4}")
print(f"Homomorphic result: {m1_times_m2_plus_m3_times_m4}")

Regular result: 232
Homomorphic result: 232


In [36]:
variables = [3, 6, 11, 8]
operations = ["mult", "add", "mult"]

result = variables[0]
for variable, operation in zip(variables[1:], operations):
    match operation:
        case "mult":
            encrypt_fun = lambda m: rsa_encrypt(m, r_pub_key)
            decrypt_fun = lambda m: rsa_decrypt(m, r_priv_key)
        case "add":
            encrypt_fun = lambda m: paillier_encrypt(m, p_pub_key)
            decrypt_fun = lambda m: paillier_decrypt(m, p_pub_key, p_priv_key)
    result_enc = encrypt_fun(result)
    variable_enc = encrypt_fun(variable)
    # Serwer: pomnóż
    new_result_enc = result_enc * variable_enc
    # Serwer: oddaj
    result = decrypt_fun(new_result_enc)

print(result)

232
